In [37]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException
import gensim
import gensim.corpora as corpora
import re
import pandas as pd
from selenium.webdriver.chrome.options import Options
from gensim.models import CoherenceModel
import nltk
import spacy
from pprint import pprint
import pyLDAvis.gensim

options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

DRIVER_PATH = 'chromedriver.exe'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
def glassdoor_review_scrape(email_val, password_val, review_page_1_url):
    driver.get('https://www.glassdoor.co.uk/profile/login_input.htm')
    email_address = driver.find_element_by_id('userEmail')
    password = driver.find_element_by_id('userPassword')
    email_address.send_keys(email_val)
    password.send_keys(password_val)
    submit_login = driver.find_element_by_name('submit').click()
    success  = WebDriverWait(driver, 10).until(presence_of_element_located((By.XPATH, "//*[contains(text(), 'Your Glassdoor Dashboard')]")))

    driver.get(review_page_1_url)
    review_id = 0
    all_records = []
    ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)

    while True:
        sleep(2) #temporary measure to avoid stale elements; needs revision
        waited  = WebDriverWait(driver, 10,ignored_exceptions=ignored_exceptions).until(expected_conditions.presence_of_element_located((By.CSS_SELECTOR, "[class^='v2__EIReviewDetailsV2__fullWidth']")))
        waited_2  = WebDriverWait(driver, 10,ignored_exceptions=ignored_exceptions).until(expected_conditions.presence_of_element_located((By.XPATH, './/div[@class="author minor"]')))
        reviews = driver.find_elements_by_class_name('gdReview')
        for review in reviews:
            review_date = review.find_element_by_xpath('.//time').text
            score = review.find_element_by_class_name('v2__EIReviewsRatingsStylesV2__ratingInfo').text
            pros = review.find_elements_by_css_selector("[class^='v2__EIReviewDetailsV2__fullWidth']")[0].text
            cons = review.find_elements_by_css_selector("[class^='v2__EIReviewDetailsV2__fullWidth']")[1].text
            review_id += 1
            job_status = review.find_element_by_xpath('.//div[@class="author minor"]').text
            employment_duration = review.find_element_by_xpath('.//p[@class="mainText mb-0"]').text
            record = [review_date, score, pros, cons, review_id, job_status, employment_duration]
            all_records.append(record)

        next_button = driver.find_element_by_xpath('//button[@class="nextButton css-sed91k"]')
        if next_button.get_property('disabled') == True:
            break
        if next_button.get_property('disabled type') == True:
            break


        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        next_button.click()
        waited_3  = WebDriverWait(driver, 10).until(presence_of_element_located((By.XPATH, '//button[@class="nextButton css-sed91k"]')))
        driver.quit()
        return all_records

all_records = glassdoor_review_scrape('*****', '*****', 'https://www.glassdoor.co.uk/Reviews/Efficio-Reviews-E220701.htm?filter.iso3Language=eng')
print(all_records[0:3])



e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['11 January 2021', '5.0\n★★★★★', 'Pros\nGood work-life balance for a consulting firm. Good salaries and rapid promotions. Amazing professionals and people. Company takes very well care of its employees.', 'Cons\nToo specialized in procurement and too much travel', 1, 'Former Employee - Consultant in London, England', 'I worked at Efficio full-time for more than 3 years'], ['6 February 2021', '3.0\n★★★★★', 'Pros\nFinally starting to move forward with modern cloud technology A good amount of freedom to make some choices like which frameworks to use A good opportunity to learn new technologies and solve challenging problems', 'Cons\nSenior leadership have little technical understanding, no CTO or head of Engineering to make decisions Due to lack of technical leadership, lots of bad short term decisions are made which increasingly build up technical debt Ceiling for technical people, you can only progress above senior level if you are like a consultant Lack of DevOps people, mostly devel

In [38]:
df = pd.DataFrame.from_records(all_records, columns=['Date', 'Rating', 'Pros', 'Cons', 'Index', 'Job title', 'Employment duration'])

df['Pros'] = df['Pros'].map(lambda x: x.lstrip('Pros\n'))
df['Cons'] = df['Cons'].map(lambda x: x.lstrip('Cons\n'))

df['Pros'].head(5)


e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0    Good work-life balance for a consulting firm. ...
1    Finally starting to move forward with modern c...
2    The people and the culture is the reason to wo...
3    - Good place to build core analytical skills (...
4                 Big clients and interesting projects
Name: Pros, dtype: object

In [39]:
df['Pros'] = df['Pros'].map(lambda x: re.sub('[,\!?]', '', x))
df['Pros'] = df['Pros'].map(lambda x: x.lower())
df.head()



nltk.download('stopwords')
from nltk.corpus import stopwords

e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:1: DeprecationWarning: invalid escape sequence \!
<>:1: DeprecationWarning: invalid escape sequence \!
<ipython-input-39-355f26184c68>:1: DeprecationWarning: invalid escape sequence \!
  df['Pros'] = df['Pros'].map(lambda x: re.sub('[,\!?]', '', x))
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Joe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
pro_sentence_list = df['Pros'].tolist()
print(pro_sentence_list[0:3])
pro_sentence_list = [re.split(r'-|\.',sentence) for sentence in pro_sentence_list]
pro_sentence_list = [point for sentence in pro_sentence_list for point in sentence if point!='']
print(pro_sentence_list[0:3])
stpword = stopwords.words('english')

def sentences_to_words(sentences):
    for i in sentences:
        yield(gensim.utils.simple_preprocess(i, deacc=True))

pro_word_list = list(sentences_to_words(pro_sentence_list))


def rm_stopwords(words_list):
    return [[word for word in words if word not in stpword] for words in words_list]

pro_word_list = rm_stopwords(pro_word_list)


['good work-life balance for a consulting firm. good salaries and rapid promotions. amazing professionals and people. company takes very well care of its employees.', 'finally starting to move forward with modern cloud technology a good amount of freedom to make some choices like which frameworks to use a good opportunity to learn new technologies and solve challenging problems', 'the people and the culture is the reason to work for efficio. despite rapid growth in recent years efficio has still managed to maintain a collaborative and tight-knit environment.']
['good work', 'life balance for a consulting firm', ' good salaries and rapid promotions']


e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

pro_word_list_lemm = lemmatization(pro_word_list, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
print(pro_word_list_lemm[0:3])

id2word = corpora.Dictionary(pro_word_list_lemm)
texts = pro_word_list_lemm
corpus = [id2word.doc2bow(text) for text in texts]

[['good', 'work'], ['life', 'balance', 'consulting', 'firm'], ['good', 'salarie', 'rapid', 'promotion']]


e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [43]:

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.042*"work" + 0.042*"company" + 0.042*"consulting" + 0.023*"environment" + '
  '0.023*"great" + 0.023*"diversity" + 0.023*"skillset" + 0.023*"cliquey" + '
  '0.023*"senior" + 0.023*"quickly"'),
 (1,
  '0.080*"good" + 0.067*"work" + 0.041*"culture" + 0.028*"project" + '
  '0.028*"place" + 0.028*"dynamic" + 0.028*"promotion" + 0.016*"challenge" + '
  '0.015*"people" + 0.015*"life"'),
 (2,
  '0.035*"amazing" + 0.035*"professional" + 0.035*"people" + 0.035*"still" + '
  '0.035*"recent" + 0.035*"collaborative" + 0.035*"year" + 0.035*"manage" + '
  '0.035*"maintain" + 0.035*"rapid"'),
 (3,
  '0.040*"technology" + 0.040*"good" + 0.022*"top" + 0.022*"big" + '
  '0.022*"procurement" + 0.022*"set" + 0.022*"interesting" + 0.022*"team" + '
  '0.022*"member" + 0.022*"great"'),
 (4,
  '0.031*"technology" + 0.031*"lot" + 0.031*"pay" + 0.031*"company" + '
  '0.031*"manager" + 0.031*"level" + 0.031*"efficio" + 0.031*"reason" + '
  '0.031*"develop" + 0.031*"suite"')]


e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
coherence_model_lda = CoherenceModel(model=lda_model, texts=pro_word_list_lemm, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -5.2961969304084775

Coherence Score:  0.43984108873593497


In [45]:

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis




e:\latest pycharm\venv\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.061097  0.128535       1        1  36.895829
3     -0.101007 -0.089150       2        1  22.061634
0      0.120076 -0.008149       3        1  20.626470
4      0.035010  0.001648       4        1  11.419280
2      0.007018 -0.032885       5        1   8.996787, topic_info=          Term      Freq     Total Category  logprob  loglift
13     company  2.000000  2.000000  Default  30.0000  30.0000
10      people  2.000000  2.000000  Default  29.0000  29.0000
0         good  6.000000  6.000000  Default  28.0000  28.0000
37     efficio  1.000000  1.000000  Default  27.0000  27.0000
34  technology  2.000000  2.000000  Default  26.0000  26.0000
..         ...       ...       ...      ...      ...      ...
36     culture  0.078092  3.128030   Topic5  -5.1522  -1.2820
2      balance  0.078092  2.291746   Topic5  -5.1522  -0.9709
13     company  0.078092  2.167492   Topic5  -5.1522  -0.9151
87    exposure  0.078091  1.847995   Topic5  -5.1522  -0.7557
6    promotion  0.078091  1.972180   Topic5  -5.1522  -0.8207

[221 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
17        2  0.864395      amount
49        1  0.793497  analytical
2         1  0.436348     balance
2         3  0.436348     balance
2         4  0.436348     balance
...     ...       ...         ...
1         1  0.698502        work
1         3  0.174625        work
1         4  0.174625        work
69        1  0.793498   workforce
70        1  0.793499       young

[117 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 5, 3])